# Trabajo Final - Planning - MAIE
En la presente notebook se realizará el trabajo final de la materia Planning de la MAIE, y se explicará paso a paso su resolución. Primero, seleccionar la fecha a procesar en la variable ***fecha*** de la forma YYYY-MM-DD en formato string

In [ ]:
# ----- Elegir fecha a procesar -----
fecha = '2023-01-06'

# ----------------------------------------------------------
# ----- NO MODIFICAR: Importación de librarías -------------
# ----------------------------------------------------------
import os
import sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from osgeo import gdal
gdal.UseExceptions()

# -----------------------------------------
# ----- NO MODIFICAR: CHEQUEO -------------
# -----------------------------------------
path = os.getcwd()

# Verifico que exista la carpeta de datos, si no lanzo error
if not os.path.exists(f'{path}/datos'):
    raise Exception(
        '''
        No existe la carpeta "datos" en el directorio actual.
        Verificar que el path de ejecución sea donde se clonó el repositorio
        ''')

# Verifico que exista la carpeta de resultados, si no la creo
if not os.path.exists(f'{path}/plots'):
    os.system(f'mkdir {path}/plots')

# Verifico que exista la fecha a procesar
# Los-Molinos_2023-01-06_0_all_bands
datos_path_list = glob.glob(f'{path}/datos/Los-Molinos_{fecha}_*_all_bands.tif')
if len(datos_path_list) == 0:
    raise Exception(f'No se encontraron datos para la fecha {fecha}')